In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


<img src="./MNIST.png" align="left" width="300"/>

MNIST 손글씨 실습을 위해서 케라스에서 제공하는 MNIST 데이터 셋을 사용한다.

학습 데이터에는 60,000개의 샘플 데이터가 있고 테스트 데이터에는 총 10,000개의 샘플 데이터가 있다.  
MNIST 손글씨 데이터는 이미지 하나가 28개의 행과 28개의 열을 가지는 픽셀 데이터이고 각 픽셀은 흑백 사진과 같이 0부터 255까지의 그레이스케일을 가지고 있다.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print('x_train.shape: {}, y_train.shape: {}'.format(x_train.shape, y_train.shape))
print('x_test.shape: {}, y_test.shape: {}'.format(x_test.shape, y_test.shape))

x_train.shape: (60000, 28, 28), y_train.shape: (60000,)
x_test.shape: (10000, 28, 28), y_test.shape: (10000,)


6만개의 학습 데이터를 학습 데이터(5만개)와 검증 데이터(1만개)로 분리한다.  
학습 중간마다 검증 데이터로 모델 성능을 측정하면 모델 학습이 제대로 진행되는지 검증 정확도를 알 수 있고 학습 정확도는 올라가는데 검증 정확도가 더 이상 올라가지 않거나 오히려 떨어질 경우 학습의 조기 종료를 구현할 수 있다.

In [3]:
# 검증 데이터로 사용하기 위해서 학습 데이터에서 1만개를 분리한다.
x_val = x_train[50000:] # 28행 28열로 구성된 검증 데이터
x_train = x_train[:50000] # 28행 28열로 구성된 학습 데이터
y_val = y_train[50000:] # 검증 데이터 레이블
y_train = y_train[:50000] # 학습 데이터 레이블
print('x_train.shape: {}, y_train.shape: {}'.format(x_train.shape, y_train.shape)) # 학습
print('x_val.shape: {}, y_val.shape: {}'.format(x_val.shape, y_val.shape)) # 학습 중 검증
print('x_test.shape: {}, y_test.shape: {}'.format(x_test.shape, y_test.shape)) # 학습 후 테스트

x_train.shape: (50000, 28, 28), y_train.shape: (50000,)
x_val.shape: (10000, 28, 28), y_val.shape: (10000,)
x_test.shape: (10000, 28, 28), y_test.shape: (10000,)


학습 데이터를 출력해보면 데이터가 0부터 255사이의 숫자(그레이스케일)로 구성된 것을 확인할 수 있다.

In [4]:
print(y_train[:10])
for i in x_train[0]:
    for j in i:
        print('{:3d} '.format(j), end='')
    print()

[5 0 4 1 9 2 1 3 1 4]
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136 175  26 166 255 247 127   0   0   0   0 
  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253 225 172 253 242 195  64   0   0   0   0 
  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82  82  56  39   0   0   0   0   0 
  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241   

다층 퍼셉트론을 사용한 손글씨 이미지 분류 작업 흐름도

<img src="./MNIST2.png" align="left" width="1300"/>

다층 퍼셉트론의 입력값은 무조건 1차원 형태의 배열만 가능하다.  
MNIST 손글시 데이터를 다층 퍼셉트론의 입력값으로 사용할 수 있도록 넘파이의 reshape() 메소드를 사용해서 2차원 배열 형태의 데이터를 1차원 배열 형태로 변경한다.

In [5]:
print('x_train.shape: {}'.format(x_train.shape))
# 28행 28열로 구성된 학습 데이터를 784개의 1차원 배열 형태로 변환한다.
# x_train = np.reshape(x_train, [50000, 784])
x_train = x_train.reshape(50000, 784)
# x_train = x_train.reshape(-1, 784)
print('x_train.shape: {}'.format(x_train.shape))
# 28행 28열로 구성된 검증 데이터를 784개의 1차원 배열 형태로 변환한다.
x_val = x_val.reshape(10000, 784)
print('x_val.shape: {}'.format(x_val.shape))
# 28행 28열로 구성된 테스트 데이터를 784개의 1차원 배열 형태로 변환한다.
x_test = x_test.reshape(10000, 784)
print('x_test.shape: {}'.format(x_test.shape))

x_train.shape: (50000, 28, 28)
x_train.shape: (50000, 784)
x_val.shape: (10000, 784)
x_test.shape: (10000, 784)


1차원으로 변경된 데이터를 그대로 다층 퍼셉트론의 입력되도 되지만 조금 더 효율적인 학습을 위해서 데이터를 정규화 한다.  
데이터를 정규화 하면 모델의 학습 시간을 단축시키고 더 나은 성능을 보이는 효과가 있다.  
MNIST 손글씨 데이터의 모든 값들은 0부터 255사이의 범위 안에 있으므로 255로 나눠 모든 값들이 0부터 1사이의 값으로 정규화 된다.

In [6]:
# MNIST 손글씨 데이터의 각 픽셀의 데이터 타입은 부호 없이 0부터 255사이의 값만 기억하면 되므로 데이터 타입이
# 부호없는 8비트 정수(uint8)로 되어있다.
# print(type(x_train[0][0])) # <class 'numpy.uint8'>
# 255로 나눠서 0부터 1사이의 실수로 만들어야 하므로 astype() 메소드로 실수로 변환한다.
x_train = x_train.astype(np.float32)
# print(type(x_train[0][0])) # <class 'numpy.float32'>
x_train /= 255
x_val = x_val.astype(np.float32)
x_val /= 255
x_test = x_test.astype(np.float32)
x_test /= 255

MNIST 손글씨 데이터 분류 모델은 0에서 9사이의 숫자로 분류하는 모델이므로 손실 함수로 크로스 엔트로피를 사용한다.  
크로스 엔트로피를 계산하기 위해서 레이블(y_train, y_val, y_test)을 원-핫 인코딩(one-hot encoding)으로 변환한다.  
원-한 인코딩은 데이터를 수많은 0과 1개의 1로 구별하는 인코딩 방식으로 0으로 이루어진 벡터 집합에서 단 1개의 1의 값으로 해당 데이터를 구별하는 것을 말한다.

In [7]:
print(set(y_train))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [8]:
# to_categorical() 메소드로 데이터에 원-핫 인코딩을 적용할 수 있다.
print(y_train[:5])
y_train = tf.keras.utils.to_categorical(y_train, 10) # 학습 데이터의 레이블에 원-핫 인코딩을 적용한다.
for i in y_train[:5]:
    print(i)
y_val = tf.keras.utils.to_categorical(y_val, 10) # 검증 데이터의 레이블에 원-핫 인코딩을 적용한다.
y_test = tf.keras.utils.to_categorical(y_test, 10) # 테스트 데이터의 레이블에 원-핫 인코딩을 적용한다.

[5 0 4 1 9]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


입력 데이터는 784개의 숫자가 들어있는 1차원 배열 형태이다.  
784개의 입력을 받는 256개의 노드가 첫 번째 히든 레이어에 있고 첫 번째 히든 레이어의 출력값을 입력으로 받는 두 번째 히든 레이어에는 128개의 노드가 있다.  
두 번째 히든 레이어에는 과대 적합을 방지하기 위해 10%의 드롭 아웃을 적용한다.  
세 번째 히든 레이어에서는 총 10개의 노드가 존재하며 이 10개의 노드값은 소프트 맥스를 통해서 0부터 0사이에 해당되는 각 숫자의 확률을 의미한다.  
소프트 맥스는 분류해야하는 레이블의 총 개수를 k라고 할 때 k차원의 벡터를 입력받아 각 정답에 대한 확률을 추정한다.  
소프트 맥스의 출력값과 실제값의 차이(오차)를 계산하기 위해 크로스 엔트로피를 손실 함수로 사용하고 손실 함수를 최소화 하기 위해서 Adam 옵티마이저(최적화 함수)를 사용해서 역전파를 통해 모든 가중치 및 편향값을 최적화 한다.

최적화 함수 참고 사이트  
https://onevision.tistory.com/entry/Optimizer-%EC%9D%98-%EC%A2%85%EB%A5%98%EC%99%80-%ED%8A%B9%EC%84%B1-Momentum-RMSProp-Adam

<img src="./MNIST3.png" align="left" width="800"/>

소프트 맥스(Soft Max)의 동작 원리

In [9]:
a = tf.constant([3, 10, 5]) # 1차원 배열
sess = tf.Session()
print(sess.run(a))
# argmax() 메소드 배열에서 가장 큰 값을 찾아서 인덱스를 리턴한다.
# a 배열에서 10이 가장 크기 때문에 결과는 10의 인덱스인 1이 출력된다.
print(sess.run(tf.argmax(a)))
sess.close()

[ 3 10  5]
1


In [10]:
a = tf.constant([[3, 10, 5], [4, 5, 6], [0, 8, 7]]) # 2차원 배열
sess = tf.Session()
print(sess.run(a))
# argmax() 메소드를 2차원에서 사용할 경우 적절한 2번째 인수를 지정해야 한다. 생략시 기본값은 0이다.
# 2번째 인수를 생략하거나 0을 사용하면 2차원 배열의 각 열에서 최대값의 인덱스를 리턴하고 2번째 인수에 1을 사용하면
# 배열의 각 행에서 최대값의 인덱스를 리턴한다.
# a 배열에서 열 단위로 최대값을 계산하면 0번째 열의 최대값인 4의 인덱스 1, 1번째 열의 최대값인 10의 인덱스 0,
# 2번째 열의 최대값인 7의 인덱스 2가 [1 0 2]와 같이 리턴된다.
print(sess.run(tf.argmax(a))) # [1 0 2]
print(sess.run(tf.argmax(a, 0))) # [1 0 2]
# a 배열에서 행 단위로 최대값을 계산하면 0번째 행의 최대값인 10의 인덱스 1, 1번째 행의 최대값인 6의 인덱스 2,
# 2번째 행의 최대값인 8의 인덱스 1이 [1 2 1]와 같이 리턴된다.
print(sess.run(tf.argmax(a, 1))) # [1 2 1]
sess.close()

[[ 3 10  5]
 [ 4  5  6]
 [ 0  8  7]]
[1 0 2]
[1 0 2]
[1 2 1]


In [11]:
# placeholder를 만들 때 2차원 이상의 데이터를 기억하는 경우 shape 속성을 이용해서 데이터의 차원을 지정해야 한다.
# x, y placeholder의 shape 속성의 첫 번째 값을 None으로 지정한 이유는 데이터 개수의 제약없이 입력받기 위해서이고
# 두 번째 속성값을 784와 10으로 지정한 이유는 MNIST 손글씨 이미지의 크기가 28 * 28 = 784 픽셀이고 학습 데이터의
# 레이블이 0에서 9까지 10개의 값이기 때문이다.
x = tf.placeholder(dtype=tf.float32, shape=[None, 784]) # 학습 데이터(x_train)를 기억할 placeholder 선언한다.
y = tf.placeholder(dtype=tf.float32, shape=[None, 10]) # 학습 데이터의 레이블(y_train)을 기억할 placeholder 선언한다.
keep_prob = tf.placeholder(dtype=tf.float32) # 드롭 아웃 값을 기억할 placeholder 선언한다.

다층 퍼셉트론을 구현한다.

In [12]:
# 히든 레이어 부분을 함수로 구현한다.
def mlp(x):
    # 히든 레이어1
    # 784개의 입력을 받는 256개의 뉴런을 사용하기 위해서 가중치를 [입력 데이터 개수, 뉴런의 개수] 만큼의 난수를 만든다.
    w1 = tf.Variable(tf.random_uniform([784, 256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1) # 히든 레이어1의 출력
    # 히든 레이어2
    # 256개의 입력을 받는 128개의 뉴런을 사용하기 위해서 가중치를 [입력 데이터 개수, 뉴런의 개수] 만큼의 난수를 만든다.
    w2 = tf.Variable(tf.random_uniform([256, 128]))
    b2 = tf.Variable(tf.zeros([128]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2) # 히든 레이어2의 출력
    # 히든 레이어2의 출력에 dropout() 메소드로 드롭 아웃을 적용한다.
    # 예전에는 rate=keep_prob 형태로 사용했었는데 버전이 올라가면서 rate=1-keep_prob와 같은 형태로 사용한다.
    h2_drop = tf.nn.dropout(h2, rate=1-keep_prob)
    # 히든 레이어3
    # 128개의 입력을 받는 10개의 뉴런을 사용하기 위해서 가중치를 [입력 데이터 개수, 뉴런의 개수] 만큼의 난수를 만든다.
    w3 = tf.Variable(tf.random_uniform([128, 10]))
    b3 = tf.Variable(tf.zeros([10]))
    # 다층 퍼셉트론의 출력값을 logit(logistic + probit)로 정의한다. probit는 확률을 재는 단위를 의미한다.
    logit = tf.nn.relu(tf.matmul(h2_drop, w3) + b3) # 히든 레이어3의 출력
    return logit

In [13]:
logit = mlp(x)
# logit와 레이블의 크로스 엔트로피를 손실 함수로 사용한다.
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=y)
)

# Adam 옵티마이저를 사용해 모델을 최적화 한다.
# 모델 최적화 과정은 모델의 예측값과 레이블의 차이를 줄여나가는 과정을 의미한다.
train = tf.train.AdamOptimizer(0.01).minimize(loss)

조기 종료는 과대 적합을 피하고 충분한 학습을 하기 위해서 학습 중간마다 검증 데이터(x_val)에 대한 정확도를 측정하면 학습 데이터에 대한 정확도는 계속 증가하는 반면에 검증 데이터에 대한 검증 정확도가 점차 떨어질 경우 학습을 중지하는 것을 말한다.  
매 epoch 마다 검증 데이터로 검증 정확도를 측정해서 검증 정확도가 5번 연속으로 검증 정확도의 최대값보다 높지 않을 경우 조기 종료를 수행한다.

In [14]:
saver = tf.train.Saver() # tensorflow에서 학습 모델의 저장 및 로드에 사용할 객체를 선언한다.
epoch_cnt = 300 # 조기 종료가 일어나지 않을 경우 최대 학습 횟수를 설정한다.
batch_size = 1000 # 1번에 읽어서 처리할 학습 데이터 개수를 설정한다.
iteration = len(x_train) // batch_size # batch_size에 따른 1 epoch 당 학습 횟수를 설정한다.
earlystop = 5 # 검증 정확도가 검증 정확도의 최대값 보다 5번 연속으로 높지 않을 경우 조기 종료하도록 설정한다.
earlystop_cnt = 0 # 검증 정확도가 검증 정확도의 최대값 보다 연속으로 높지 않은 검증 횟수를 세는 변수를 선언한다.

학습을 진행한다.

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    prev_train_acc = 0.0 # 이전 학습 정확도를 기억할 변수를 선언한다.
    max_val_acc = 0.0 # 검증 정확도의 최대값을 기억할 변수를 선언한다.
    # 지정한 최대 epoch 만큼 반복하며 학습한다.
    # 검증 정확도가 검증 정확도의 최대값보다 5번 연속으로 높지 않을 경우 조기 종료한다.
    for epoch in range(epoch_cnt):
        avg_loss = 0.0 # epoch 당 손실 값을 기억할 변수를 선언한다.
        start = 0 # 학습 시작 위치
        end = batch_size # 학습 종료 위치
        
        # 1 epoch 학습시 학습 데이터를 batch_size개 만큼씩 나눠서 학습을 진행한다.
        for i in range(iteration):
            _, loss_ = sess.run([train, loss], feed_dict={x: x_train[start:end], y: y_train[start:end], keep_prob: 0.9})
            # 학습할 데이터의 범위를 batch_size개 만큼 이동시킨다.
            start += batch_size
            end += batch_size
            # 크로스 엔트로피 손실 함수의 학습 손실을 계산한다.
            avg_loss += loss_ / iteration
        # ===== for i
        
        # 1 epoch 학습시 모델 검증을 진행한다.
        # 소프트 맥스를 적용해서 예측한다.
        predict = tf.nn.softmax(logits=logit)
        # 정확도를 계산하는 수식을 만든다.
        correct_predict = tf.equal(tf.argmax(predict, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_predict, 'float'))
        
        # Session().run()와 Tensor.eval()의 차이
        # t가 Tensor 객체라면 t.eval()은 sess.run(t)의 속기 표현이다. 단, sess가 현재 세션인 곳에서만 가능하다.
        # 학습 정확도를 계산한다.
        cur_train_acc = accuracy.eval({x: x_train, y: y_train, keep_prob: 1.0})
        # 검증 정확도를 계산한다.
        cur_val_acc = accuracy.eval({x: x_val, y: y_val, keep_prob: 1.0})
        # epoch 당 학습 정확도와 검증 정확도를 출력한다.
        print('epoch: {:3d}, 학습 정확도: {:7.5f}, 검증 정확도: {:7.5f}'.format(epoch + 1, cur_train_acc, cur_val_acc))
        
        # 최대 검증 정확도와 현재 검증 정확도를 비교한다.
        if cur_val_acc < max_val_acc:
            # 현재 검증 정확도가 최대 검증 정확도 미만이면
            # 현재 학습 정확도와 이전 학습 정확도, 현재 학습 정확도와 0.99를 비교한다.
            if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                # 현재 학습 정확도가 이전 학습 정확도 보다 크거나 현재 학습 정확도가 0.99 보다 크면
                if earlystop_cnt == earlystop:
                    print('조기 종료 시점: {} epoch'.format(epoch))
                    print('현재 검증 정확도가 최대 검증 정확도 보다 연속으로 {}번 작게나와서 조기 종료가 실행됨'.format(earlystop))
                    break
                else:
                    # 5번 연속으로 현재 학습 정확도가 이전 학습 정확도 보다 크지 않다면 이전 학습 정확도가
                    # 현재 학습 정확도 보다 크므로 현재 검증 정확도가 최대 검증 정확도 보다 연속으로 크지 않은 횟수를
                    # 카운트 하는 변수를 1증가 시킨다.
                    earlystop_cnt += 1
                    print('과대 적합 경고 횟수: {}'. format(earlystop_cnt))
                # ===== if earlystop_cnt == earlystop:
            else:
                # 현재 학습 정확도가 이전 학습 정확도 이하이고 현재 학습 정확도가 0.99 이하이면
                # 이전 학습 정확도가 현재 학습 정확도 이상이므로 현재 검증 정확도가 검증 정확도의 최대값보다
                # 연속으로 높지 않은 횟수를 카운트 하는 변수를 0으로 초기화 시킨다.
                earlystop_cnt = 0
            # ===== if cur_train_acc > prev_train_acc
        else:
            # 현재 검증 정확도가 최대 검증 정확도 이상이면
            # 현재 검증 정확도가 최대 검증 정확도 보다 연속으로 높지 않은 횟수를 기억하는 변수를 0으로
            # 초기화시키고 최대 검증 정확도를 현재 검증 정확도로 교체한다.
            earlystop_cnt = 0
            max_val_acc = cur_val_acc
            # 검증 정확도가 가장 높은 모델을 저장한다.
            saver.save(sess, './model/model.ckpt')
        # ===== if cur_val_acc < max_val_acc:
        # 다음 epoch를 위해 현재 학습 정확도를 이전 학습 정확도를 기억하는 변수에 넣어준다.
        prev_train_acc = cur_train_acc
    # ===== for epoch
# ===== with

epoch:   1, 학습 정확도: 0.21354, 검증 정확도: 0.21720
epoch:   2, 학습 정확도: 0.52822, 검증 정확도: 0.54310
epoch:   3, 학습 정확도: 0.60704, 검증 정확도: 0.63260
epoch:   4, 학습 정확도: 0.67774, 검증 정확도: 0.69820
epoch:   5, 학습 정확도: 0.71844, 검증 정확도: 0.73790
epoch:   6, 학습 정확도: 0.74798, 검증 정확도: 0.76840
epoch:   7, 학습 정확도: 0.77262, 검증 정확도: 0.79060
epoch:   8, 학습 정확도: 0.79344, 검증 정확도: 0.80770
epoch:   9, 학습 정확도: 0.80988, 검증 정확도: 0.82120
epoch:  10, 학습 정확도: 0.82152, 검증 정확도: 0.83330
epoch:  11, 학습 정확도: 0.83512, 검증 정확도: 0.84500
epoch:  12, 학습 정확도: 0.84356, 검증 정확도: 0.85190
epoch:  13, 학습 정확도: 0.85210, 검증 정확도: 0.85990
epoch:  14, 학습 정확도: 0.85980, 검증 정확도: 0.86700
epoch:  15, 학습 정확도: 0.86592, 검증 정확도: 0.87330
epoch:  16, 학습 정확도: 0.87228, 검증 정확도: 0.87840
epoch:  17, 학습 정확도: 0.87778, 검증 정확도: 0.88310
epoch:  18, 학습 정확도: 0.88322, 검증 정확도: 0.88750
epoch:  19, 학습 정확도: 0.88680, 검증 정확도: 0.89000
epoch:  20, 학습 정확도: 0.89110, 검증 정확도: 0.89410
epoch:  21, 학습 정확도: 0.89450, 검증 정확도: 0.89580
epoch:  22, 학습 정확도: 0.89836, 검증 정확도: 0.90000
epoch:  23

epoch: 167, 학습 정확도: 0.98808, 검증 정확도: 0.95930
epoch: 168, 학습 정확도: 0.99014, 검증 정확도: 0.95990
epoch: 169, 학습 정확도: 0.98798, 검증 정확도: 0.95880
epoch: 170, 학습 정확도: 0.98986, 검증 정확도: 0.95910
과대 적합 경고 횟수: 1
epoch: 171, 학습 정확도: 0.98962, 검증 정확도: 0.95750
epoch: 172, 학습 정확도: 0.98846, 검증 정확도: 0.96060
epoch: 173, 학습 정확도: 0.98912, 검증 정확도: 0.95960
과대 적합 경고 횟수: 1
epoch: 174, 학습 정확도: 0.98930, 검증 정확도: 0.95810
과대 적합 경고 횟수: 2
epoch: 175, 학습 정확도: 0.98962, 검증 정확도: 0.95810
과대 적합 경고 횟수: 3
epoch: 176, 학습 정확도: 0.98804, 검증 정확도: 0.95860
epoch: 177, 학습 정확도: 0.98904, 검증 정확도: 0.95780
과대 적합 경고 횟수: 1
epoch: 178, 학습 정확도: 0.98912, 검증 정확도: 0.95760
과대 적합 경고 횟수: 2
epoch: 179, 학습 정확도: 0.98888, 검증 정확도: 0.95690
epoch: 180, 학습 정확도: 0.98870, 검증 정확도: 0.95740
epoch: 181, 학습 정확도: 0.99004, 검증 정확도: 0.96190
epoch: 182, 학습 정확도: 0.99158, 검증 정확도: 0.96110
과대 적합 경고 횟수: 1
epoch: 183, 학습 정확도: 0.99104, 검증 정확도: 0.95870
과대 적합 경고 횟수: 2
epoch: 184, 학습 정확도: 0.99096, 검증 정확도: 0.95890
과대 적합 경고 횟수: 3
epoch: 185, 학습 정확도: 0.99236, 검증 정확도: 0.95960
과대 적합 경고 횟

검증 결과가 가장 높았언 모델을 불러온다.

In [16]:
with tf.Session() as sess:
    saver.restore(sess, './model/model.ckpt')
    predict = tf.nn.softmax(logits=logit)
    correct_predict = tf.equal(tf.argmax(predict, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predict, 'float'))
    correct_predict = tf.equal(tf.argmax(predict, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predict, 'float'))
    print('학습 정확도: {:7.5f}'.format(accuracy.eval({x: x_train, y: y_train, keep_prob: 1.0})))
    print('테스트 정확도: {:7.5f}'.format(accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0})))

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
학습 정확도: 0.99004
테스트 정확도: 0.95700
